In [ ]:
import pandas as pd
import sklearn
from utils import SQLHandler
import numpy as np

from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch

from sklearn.metrics import log_loss, roc_auc_score, recall_score

import warnings
warnings.filterwarnings("ignore")

handler = SQLHandler()

# 导入数据

In [ ]:
sql = """
select *
from data_hb.e_jir_sample_0830_zyf
"""

df = handler.query(sql)

In [ ]:
sql ="""
select *
from data_hb.e_jir_sample_0830_appList

"""
df_app = handler.query(sql)

In [ ]:
sql = """
select *
from data_hb.e_jir_sample_0830_xd

"""
df_xd = handler.query(sql)

In [ ]:
sql = """
SELECT *
from
data_hb.e_jir_sample_0830_location
  """
df_location = handler.query(sql)

In [ ]:
sql = """
select *
from
(select *, row_number() over(partition by uid order by date desc) as r
from data_hb.e_jir_sample_0830_xd
)t
where r = 1
"""
df_xd2 = handler.query(sql)

In [ ]:
sql = """
select *
from
data_hb.e_jir_sample_0830_act_province

"""

df_ac_city = handler.query(sql)

In [ ]:
sql = """
select *
from 
data_hb.e_jirong_sample_0830_f2
"""

df_f2 = handler.query(sql)

# 数据处理，特征衍生

## 申请记录数据处理

### 两类小贷产品xd1, xd2。 获取申请前90天这两类小贷产品的多头和占比

In [ ]:
df_xd_s = pd.DataFrame(df_xd.groupby(['uid','p_label'])['level'].count()).reset_index()
df_xd_s = pd.pivot(df_xd_s, index = 'uid', columns = 'p_label', values = 'level')
df_xd_s.fillna(0,inplace=True)
df_xd_s.reset_index(inplace=True)
df_xd_s = df_xd_s.merge(df_xd2[['uid','year','level','p_operate','phone_model','date','bl_flag','ex_flag','soc_flag']].drop_duplicates('uid'),on = 'uid')
df_xd_s.fillna(-1,inplace=True)
df_xd_s['xd1_per'] = df_xd_s['xd1_flag'] / (df_xd_s['xd1_flag'] + df_xd_s['xd2_flag'] )


#合并到主表
df = df[['uid','age','level','p_operate','phone_model']].merge(df_xd_s[['uid','xd1_flag','xd2_flag','bl_flag','ex_flag','soc_flag','xd1_per']],how='left',on='uid')
df[['xd1_flag','xd2_flag']] = df[['xd1_flag','xd2_flag']].fillna(0)
df.fillna(-1,inplace=True)


df = df.merge(df_xd2[['uid','p_label']], on ='uid',how = 'left')


## 地理位置数据，用城市等级编码，加入活跃城市和活跃指数

In [ ]:
city_rank = """北京、上海、广州、深圳
成都、重庆、杭州、武汉、西安、南京、天津、苏州、长沙、沈阳、青岛、济南、合肥、东莞、昆明、福州、无锡、厦门、哈尔滨、长春、南昌、济南、宁波、大连、贵阳、温州、石家庄、泉州、南宁、金华、常州、珠海、惠州、嘉兴、南通、中山、保定、兰州、台州、徐州、太原、绍兴、烟台、廊坊
海口、汕头、潍坊、扬州、洛阳、乌鲁木齐、临沂、唐山、镇江、盐城、湖州、赣州、漳州、揭阳、江门、桂林、邯郸、泰州、济宁、呼和浩特、咸阳、芜湖、三亚、阜阳、淮安 遵义、银川、衡阳、上饶、柳州、淄博、莆田、绵阳、湛江、商丘、宜昌、沧州、连云港、南阳、蚌埠、驻马店、滁州、邢台、潮州、秦皇岛、肇庆、荆州、周口"""


city_l = city_rank.split('\n')
l = []
for i in city_l:
    l.append(i.split('、'))
di = {}
for i in l[0]:
    di[i] = "一线"
for i in l[1]:
    di[i] = "二线"
for i in l[2]:
    di[i] = "三线"
    
df_location['city_rank'] = df_location['city'].map(di)
df_location.fillna('四线',inplace=True)    

df = df.merge(df_location[['uid','city_rank']],how = 'left',on = 'uid')

#活跃城市
df= df.merge(df_ac_city[['uid','rate_same_province','max_rate_act_province']], on ='uid', how = 'left')

## 其他数据

In [ ]:
#这张表包含小贷产品注册数，信用卡持卡数，小微企业主标签，信用卡申请标签
df = df.merge(df_f2, on ='uid',how = 'left')

## app列表特征衍生

### tf-idf

In [ ]:
#基于bad rate计算每个app的tf-idf。 tf为 app的未通过人数 / 总未通过人数； idf为 log(总人数 / 安装app的人数)

total_user = 56956 + 32533
bad_user = 56956
df_app3 = df_app.merge(df[['uid','label']],on='uid')
df_app_bad = pd.DataFrame(df_app3.groupby('brand')['label'].sum())
df_app_bad.columns=['bad_count']
df_app_bad['tf'] = df_app_bad['bad_count'] / bad_user
df_app_all = pd.DataFrame(df_app3.groupby('brand')['label'].count())
df_app_bad = df_app_bad.merge(df_app_all,on = 'brand')
df_app_bad['idf'] = np.log(total_user / df_app_bad['label'])
df_app_bad['tf-idf'] = df_app_bad['tf'] * df_app_bad['idf']


tfidf_dict = df_app_bad['tf-idf'].to_dict()
df_app3['tfidf'] = df_app3['brand'].map(tfidf_dict)
df_tfidf = pd.DataFrame(df_app3.groupby('uid')['tfidf'].apply(list))
df_tfidf = df_tfidf.reset_index().merge(df,on='uid')


#计算 1: 每个用户安装的app的平均tfidf； 2：每个用户安装app中tfidf大于0.25的数量
df_tfidf['score_mean'] = df_tfidf['tfidf'].apply(lambda x: np.mean(x))
def countRiskApp(x):
    c = 0
    for i in x:
        if i >= 0.25:
            c+=1
    return c
df_tfidf['risk_count'] = df_tfidf['tfidf'].apply(countRiskApp)
df = df.merge(df_tfidf[['uid','score_mean','risk_count']], on = 'uid', how = 'left')

# 特征编码， 加入app_list

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_app['brand'] = le.fit_transform(df_app['brand'])


df_app2 = pd.DataFrame(df_app.groupby('uid')['brand'].apply(list))
df_app2.reset_index(inplace=True)
df = df.merge(df_app2[['uid','brand']], how = 'left',on = 'uid')

In [ ]:
df['label'] = df.apply(lambda x: 0 if (x['level'] == 'A') or (x['level'] =='B') else 1,axis = 1)
df['age'] = 2023 - df['year'].fillna(-1).astype(int)
df['age'] = df['age'].replace(2024,df['age'].mean())

In [ ]:
cate_features_names = ['p_operate','phone_model','bl_flag','ex_flag','soc_flag','city_rank','p_label','busi','card']
num_features_names = ['age','rate_same_province','max_rate_act_province','registered_product_count','dt_num','score_mean','risk_count','xd1_flag','xd2_flag','xd1_per']
for f in cate_features_names:
    le = LabelEncoder()
    df[f] = df[f].astype('str')
    df[f] = le.fit_transform(df[f])


from sklearn.model_selection import train_test_split
X = df.drop(['label'],axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

# app列表特征衍生2

### 对app列表做两种衍生 1：one-hot编码，维度为(#user, #total_app); 2.app做label-encoding形成列表， 维度(#user, 用户有的最多app数量)

In [ ]:
def get_coo_indexes(lil):
    rows=[]
    cols=[]
    for i,el in enumerate(lil):
        if type(el) != list:
            el = [el]
            
        for j in el:
            rows.append(i)
            cols.append(j)
    return rows, cols

from scipy.sparse import coo_matrix

def get_sparse_features(series, shape):
    coo_indexes = get_coo_indexes(series.tolist())
    sparse_df = coo_matrix((np.ones(len(coo_indexes[0])), (coo_indexes[0], coo_indexes[1])),shape=shape)
    
    return sparse_df
def sparse_to_idx(df, pad_idx = -1):
    indexes = df.nonzero()
    indexes_df = pd.DataFrame()
    indexes_df['rows'] = indexes[0]
    indexes_df['cols'] = indexes[1]
    mdf = indexes_df.groupby('rows').apply(lambda x: x['cols'].tolist())
    max_len = mdf.apply(lambda x: len(x)).max()
    return mdf.apply(lambda x: pd.Series(x +[pad_idx] * (max_len - len(x)))).values

prev_brand_train = get_sparse_features(X_train['brand'], (len(df),df_app['brand'].max()+1))
prev_brand_test = get_sparse_features(X_test['brand'], (len(df),df_app['brand'].max()+1))

brand_train_tensor = torch.sparse_coo_tensor(indices = prev_brand_train.nonzero(),
                                            values = [1] * len(prev_brand_train.nonzero()[0]),
                                            size = prev_brand_train.shape).to_dense()


brand_test_tensor = torch.sparse_coo_tensor(indices = prev_brand_test.nonzero(),
                                            values = [1] * len(prev_brand_test.nonzero()[0]),
                                            size = prev_brand_test.shape).to_dense()

brand_idx_train = torch.Tensor(sparse_to_idx(prev_brand_train,0)).long()
brand_idx_test = torch.Tensor(sparse_to_idx(prev_brand_test,0)).long()

target_train = torch.Tensor(y_train.values).long()
target_valid = torch.Tensor(y_test.values).long()

# 创建dataset

In [ ]:
class build_dataset(Dataset):
    def __init__(self, dataf, cate_feature_names, num_feature_names, brand_tensor, brand_idx,target):
        self.df = dataf
        self.cate_feature_names = cate_feature_names
        self.num_feature_names = num_feature_names
        self.brand_tensor = brand_tensor
        self.brand_idx = brand_idx
        self.target = target
        
    def __getitem__(self, idx):
        idx2 = [idx]
        #print(self.num_feature_names)
        #print(self.df.iloc[idx2][self.num_feature_names])
        continuous_features = self.df.iloc[idx2][self.num_feature_names].values
       
        cate_features = self.df.iloc[idx2][self.cate_feature_names].values
        
        brand_tensor = self.brand_tensor[idx]
        brand_idx = self.brand_idx[idx]
        label = self.target[idx]
        
        return  torch.from_numpy(continuous_features), torch.from_numpy(cate_features), torch.tensor(brand_tensor),torch.tensor(brand_idx), label
    
    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = build_dataset(X_train, cate_features_names, num_features_names, brand_train_tensor,brand_idx_train, target_train)
train_loader = DataLoader(dataset = train_dataset, batch_size= 128, shuffle = True)

valid_dataset = build_dataset(X_test, cate_features_names, num_features_names, brand_test_tensor,brand_idx_test, target_valid)
valid_loader = DataLoader(dataset = valid_dataset, batch_size= 128, shuffle = False)

In [ ]:
cate_vocab_size = []
for i in cate_features_names:
    cate_vocab_size.append(df[i].nunique())
print(cate_vocab_size)

### 改进wide and deep 模型， 使用两个deep层分开训练。 app_index列表单独训练。 wide层为app_onehot，为了记忆app之间的直接关系

In [ ]:
from torch import cat, mean
class WideAndDeep(nn.Module):
    def __init__(
        self, 
        #continuous_feature_shape,
        embed_size, # size of embedding for binary features
        embed_dict_len,
        cate_vocab_size,# number of unique binary features
        num_size,
        pad_idx # padding index
    ):
        super(WideAndDeep, self).__init__()
        self.embed = nn.Embedding(embed_dict_len, embed_size, padding_idx=pad_idx)
        self.embed_layer_list = []
        for i in cate_vocab_size:
            self.embed_layer_list.append(nn.Embedding(i,embed_size))
        
        self.embed_layer_list = nn.ModuleList(self.embed_layer_list)
       
        self.deep1 = nn.Sequential(
            nn.Linear(embed_size * len(cate_vocab_size) + num_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),  
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
          
        )
        
        self.deep2 = nn.Sequential(
            nn.Linear(embed_size, 512),
            nn.ReLU(),  
            nn.Linear(512, 256),
            nn.ReLU()
        )
        
        self.wide = nn.Sequential(   
            nn.Linear(embed_dict_len + 512, 1)
        )
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, binary, binary_idx, cate_features, num_features):
        
        emb_out_list = []
        cate_features = cate_features.squeeze()
        num_features = num_features.squeeze()
        
        for i, emb_layer in enumerate(self.embed_layer_list):
            
            embed_out = emb_layer(cate_features[:,i].long())
            emb_out_list.append(embed_out)
            
        cate = torch.cat(emb_out_list, dim=1)
      
        binary_embed = self.embed(binary_idx)
        binary_embed_mean = mean(binary_embed, dim=1)
        
        
        
        x = torch.cat((cate, num_features),dim=1)
        
        deep1 = self.deep1(torch.tensor(x,dtype=torch.float32))
        
        deep2 = self.deep2(torch.tensor(binary_embed_mean))

        total_logits = self.wide(torch.cat((deep1, deep2, binary),dim=1))
        
      

        output = self.sigmoid(total_logits)
        
        
        return output

# 训练部分

In [ ]:
device = 'cpu'
model = WideAndDeep(8,df_app['brand'].nunique(),cate_vocab_size,len(num_features_names), 0).to(device)
loss_fcn = nn.BCELoss()
#loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.05, weight_decay = 0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,8,15,20,30,40], gamma=0.5)

In [ ]:
best_auc = 0
epochs = 40
for e in range(epochs):
    train_labels = []
    train_preds = []

    train_loss_sum = 0.0
    current_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print(f'Current lr: {current_lr}')
    for idx, x in enumerate(train_loader):
        model.train()       

        num_features, cate_features, brand_train_tensor, brand_train_idx, label = x[0].to(device), x[1].to(device), x[2].to(device), x[3].to(device), x[4].to(device)

        pred = model(brand_train_tensor, brand_train_idx, cate_features, num_features)
        
    
        loss = loss_fcn(pred.squeeze().to(torch.float32), label.squeeze().to(torch.float32))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        train_labels.extend(label.squeeze().cpu().clone().detach().numpy())
        train_preds.extend(pred.squeeze().cpu().clone().detach().numpy())
        
      

        train_loss_sum += loss.cpu().item()
        if(idx+1) % 20 ==0:
            print(f"loss: {train_loss_sum / (idx + 1)}")
            print(f"train auc: {roc_auc_score(train_labels, train_preds)}")
            
    scheduler.step()
    print('eval--------------------------')
    
    with torch.no_grad():
        valid_labels = []
        valid_preds = []
        for idx, x in enumerate(valid_loader):
            model.train()       

            num_features, cate_features, brand_train_tensor, brand_train_idx, label = x[0].to(device), x[1].to(device), x[2].to(device), x[3].to(device), x[4].to(device)

            pred = model(brand_train_tensor, brand_train_idx, cate_features, num_features)

            valid_labels.extend(label.squeeze().cpu().detach().numpy())
            valid_preds.extend(pred.squeeze().cpu().detach().numpy())

        auc = roc_auc_score(valid_labels, valid_preds)
        print(f"auc: {auc}")
        if auc > best_auc:
            best_auc = auc
            print(f'model_saved,auc：{auc}')
            torch.save(model.state_dict(),'model_credit_v4.pth')